In [4]:
import os
from langchain_ollama import OllamaLLM
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import DocArrayInMemorySearch # do I need permanent storage like pinecone?
from langchain_ollama import OllamaEmbeddings
from operator import itemgetter


MODEL = "gemma3"
MODEL_EMBED = "embeddinggemma"
DOCUMENTS = "resume2025.pdf"

In [5]:
# Instanciate & test model
model = OllamaLLM(model=MODEL)
model.invoke("Don't think of an elephant")

'Okay! I’m not thinking of an elephant. 😊 \n\nIs there anything you *are* thinking about, or would you like me to do something else?'

In [6]:

loader = PyPDFLoader(DOCUMENTS)
pages = loader.load_and_split()


In [7]:


template = '''
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
'''

prompt = PromptTemplate.from_template(template)
prompt.format(context="I need a Data Scientist", question="Is this a good fit")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: I need a Data Scientist\n\nQuestion: Is this a good fit\n'

In [8]:
chain = prompt | model 

In [9]:
chain.input_schema.schema()

/var/folders/z2/qtr66ng91gj6czktr43rbl4h0000gn/T/ipykernel_42747/3226659032.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  chain.input_schema.schema()


{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [10]:


embeddings = OllamaEmbeddings(model=MODEL_EMBED)

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

/Users/cindyreiner/Documents/Code/local-LLM/.venv/lib/python3.13/site-packages/docarray/helper.py:255: SyntaxWarning: invalid escape sequence '\*'
  e.g. '\*.py', '[\*.zip, \*.gz]'


In [11]:
retriever = vectorstore.as_retriever()

In [12]:


chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | prompt
    | model
    
)

print(chain.invoke({"question": "what soft skills are evident"}))
print(chain.invoke({"question": "would she be a good fit for our data scientist role"}))

Based on the resume content, the following soft skills are evident:

*   **Cross-functional Leadership:** Demonstrated in her roles at Booz Allen Hamilton and Jovial.
*   **Stakeholder Management:** Highlighted in multiple roles, particularly at Booz Allen Hamilton and Jovial.
*   **Technical Writing & Editing:** Mentioned as a key skill.
*   **Communication:** Implicit through her presentations, technical documentation, and collaboration.
*   **Grant Writing:** Explicitly listed as a skill.
*   **Research Methodology:** Demonstrated throughout her academic and research experience.
*   **Presentations:** Mentioned as a key skill.
Based on the provided resume, Cindy Reiner appears to be a strong fit for a data scientist role. Here’s a breakdown of why:

*   **Relevant Experience:** She has 8+ years of experience in quantitative analysis and research, combined with 4 years of machine learning implementation.
*   **Technical Skills:** She possesses a strong skillset including Python, SQL,